In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import numpy as np
import random
import cv2
import os
import glob


In [3]:
data = []
labels = []

learning_rate = 1e-3
img_dims = (96,96,3)
epochs = 100

# loading image files from the dataset
image_files = [f for f in glob.glob(r'C:\Users\Oggy\Downloads\Gender-Detection-master\Gender-Detection-master\gender_dataset_face' + "/**/*", recursive=True) if not os.path.isdir(f)]
random.shuffle(image_files)

# converting images to arrays and labelling the categories
for img in image_files:

    image = cv2.imread(img)
    
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2] # C:\Files\gender_dataset_face\woman\face_1162.jpg
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label]) # [[1], [0], [0], ...]

# pre-processing
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)

trainY = to_categorical(trainY, num_classes=2) # [[1, 0], [0, 1], [0, 1], ...]
testY = to_categorical(testY, num_classes=2)

# augmenting datset 
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

# define model
def build(width, height, depth, classes):
    model = Sequential()
    inputShape = (height, width, depth)
    chanDim = -1

    if K.image_data_format() == "channels_first": #Returns a string, either 'channels_first' or 'channels_last'
        inputShape = (depth, height, width)
        chanDim = 1
    
    # The axis that should be normalized, after a Conv2D layer with data_format="channels_first", 
    # set axis=1 in BatchNormalization.

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model

# build model
model = build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                            classes=2)

# compile the model
opt = Adam(learning_rate=learning_rate, decay=learning_rate/epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

# train the model
H = model.fit(aug.flow(trainX, trainY, batch_size=64),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // 64,
                        epochs=epochs, verbose=1)

# save the model to disk
model.save('gender_detection.model')

Epoch 1/20
28/28 [==============================] - 16s 541ms/step - loss: 0.9651 - accuracy: 0.7400 - val_loss: 0.7118 - val_accuracy: 0.5606
Epoch 2/20
28/28 [==============================] - 17s 604ms/step - loss: 0.5243 - accuracy: 0.7850 - val_loss: 0.5292 - val_accuracy: 0.7597
Epoch 3/20
28/28 [==============================] - 26s 944ms/step - loss: 0.4411 - accuracy: 0.8394 - val_loss: 0.5264 - val_accuracy: 0.7814
Epoch 4/20
28/28 [==============================] - 23s 808ms/step - loss: 0.4168 - accuracy: 0.8377 - val_loss: 0.4976 - val_accuracy: 0.8225
Epoch 5/20
28/28 [==============================] - 18s 642ms/step - loss: 0.3770 - accuracy: 0.8613 - val_loss: 0.5369 - val_accuracy: 0.7013
Epoch 6/20
28/28 [==============================] - 17s 621ms/step - loss: 0.3570 - accuracy: 0.8608 - val_loss: 0.5348 - val_accuracy: 0.6797
Epoch 7/20
28/28 [==============================] - 16s 567ms/step - loss: 0.3721 - accuracy: 0.8568 - val_loss: 0.5038 - val_accuracy: 0.7511